In [1]:
import pickle
import pandas as pd

In [2]:
with open("df_all_cleaned.pickle", "rb") as read_file:
    df_all = pickle.load(read_file)

In [3]:
from p05_nlp import vectorize_tfidf
from p05_nlp import reduce_dim_lsa
from p05_nlp import reduce_dim_lda
from p05_nlp import reduce_dim_nmf
from p05_nlp import display_topics

In [4]:
import re

# Indicate pd series to model

In [5]:
df_all.head()

,title,year,director,reviews_list,imdb_url,wiki_page,url,poster_url,reviews_clean
0,200 Motels,1971,Tony Palmer,"There is no film quite like 200 Motels, but a ...",https://www.imdb.com/title/tt0066732/,https://en.wikipedia.org/wiki/200_Motels,/wiki/200_Motels,upload.wikimedia.org/wikipedia/en/thumb/8/85/2...,like motel lot strange appearance ...
1,2001: A Space Odyssey,1968,Stanley Kubrick,"A stand-alone monument in cinema history, Stan...",https://www.imdb.com/title/tt0062622/,https://en.wikipedia.org/wiki/2001:_A_Space_Od...,/wiki/2001:_A_Space_Odyssey_(film),upload.wikimedia.org/wikipedia/en/thumb/9/94/S...,stand monument cinema hi stanley kubrick m...
2,37°2 le Matin (Betty Blue),1986,Jean-Jacques Beineix,The Betty of the title is like a shooting star...,https://www.imdb.com/title/tt0090563/,https://en.wikipedia.org/wiki/Betty_Blue,/wiki/Betty_Blue,upload.wikimedia.org/wikipedia/en/thumb/9/9d/B...,betty title like shoot star run hot bright...
3,42nd Street,1933,Lloyd Bacon,Most of the negative comments posted below see...,https://www.imdb.com/title/tt0024034/,https://en.wikipedia.org/wiki/42nd_Street_(film),/wiki/42nd_Street_(film),upload.wikimedia.org/wikipedia/en/thumb/8/8b/F...,negative comment post people don t like musica...
4,The 7th Voyage of Sinbad,1957,Nathan H. Juran,"Arguably, Harryhausen's finest moment. I can't...",https://www.imdb.com/title/tt0051337/,https://en.wikipedia.org/wiki/The_7th_Voyage_o...,/wiki/The_7th_Voyage_of_Sinbad,upload.wikimedia.org/wikipedia/en/thumb/a/a6/S...,arguably harryhausen fine moment t hea...


In [6]:
reviews = df_all['reviews_clean'].copy()
type(reviews)

pandas.core.series.Series

# Additional text cleaning steps

Remove names

In [7]:
names = ['allen','woody','bakshi','thomerson',
         'jack','corman','roger','karloff',
         'stanley','kubrick','harryhausen',
         'tim','helen','megan','lena','mel',
         'brooks','werner','herzog', 'william',
         'wyler','matthew','hopkins','harry',
         'bromley','davenport','dorothy','judy',
         'garland','warhol','leone','sandler',
         'adam','carpenter','warwick','davis',
         'de','john','davis','woo','kevin',
         'smith','fonda','jay','russell',
         'cronenberg','bergman','reeve','lynch',
         'palma','max','diane','cusack',
         'michael','myers','ad','th','kieslowski',
         'aniston','meyer','billy','jennifer',
         'troma','van','damme', 'jean','bruce',
         'lee','charles','george','romero',
         'tarantino','trancer','deth','verhoeven',
         'polanski','les','michel']
rem_names =  lambda x: re.sub("|".join(names), "", x)
reviews = reviews.map(rem_names)
            

Correct typos

In [8]:
reviews = reviews.map(lambda x: re.sub('pyon', 'python', x))


Remove duplicate whitespaces

In [9]:
# result = re.sub(r'\s+', ' ',   input)           # Eliminate duplicate whitespaces
rem_empty_space =  lambda x: re.sub(r'\s+', ' ', x)
reviews = reviews.map(rem_empty_space)

Create custom bi-grams

In [10]:
reviews = reviews.map(lambda x: re.sub('monty python', 'monty_python', x))
reviews = reviews.map(lambda x: re.sub('bruce lee', 'bruce_lee', x))
reviews = reviews.map(lambda x: re.sub('sci fi', 'sci_fi', x))
reviews = reviews.map(lambda x: re.sub('kung fu', 'kung_fu', x))
reviews = reviews.map(lambda x: re.sub('spaghetti western', 'spaghetti_western', x))


In [11]:
# address = "123 north anywhere street"

# for word, initial in {"NORTH":"N", "SOUTH":"S" }.items():
#     address = address.replace(word.lower(), initial)
# print address

bigrams_dict = {'kung fu':'kung_fu','martial art':'martial_arts','special effect':'special_effect',
               'monty python':'monty_python','bruce lee':'bruce_lee',
               'sci fi':'sci_fi','spaghetti western':'spaghetti_western'}

# create_bigrams = lambda x: str.replace(key, val for key, val in bigrams_dict)
def create_bigrams(review):
    for key, val in bigrams_dict.items():
        review = review.replace(key, val)
        return review
        
reviews = reviews.map(create_bigrams)

In [12]:
reviews

0        like motel lot strange appearance especially ...
1       stand monument cinema hi magnum opus space ody...
2       betty title like shoot star run hot bright bur...
3       negative comment post people don t like musica...
4       arguably fine moment t he nominate venture fan...
5       morn man recall love relationship case song sh...
6        isn t people lot general course particular br...
7        sensational infamous clockwork orange impossi...
8       don t blame state unrstand superlative surroun...
9       love ia coffin joe joseph grave love output lo...
10      probably good horror comedy doesn t fun monste...
11      dortmund mining company shut young people unem...
12       probably horrible seriously fortunately didn ...
13      m guess everybody know opportunity great simpl...
14       diction original vampire fact t ink like abel...
15      people plot sense prevertesque way mainly link...
16      wasn t feel good today know finally chance sit...
17      ve re 

# Vectorize with Tf-Idf

In [13]:
reviews_sparse_matrix_lsa, reviews_feature_names_lsa = vectorize_tfidf(reviews, min_df=7, max_df=0.8)

       aa       aan  aaron        ab  aback   abandon  abandonment     abba  \
0     0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
1     0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
2     0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
3     0.0  0.000000    0.0  0.000000    0.0  0.004250          0.0  0.00000   
4     0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
5     0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
6     0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
7     0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
8     0.0  0.000000    0.0  0.016791    0.0  0.007285          0.0  0.00000   
9     0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
10    0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
11    0.0  0.000000    0.0  0.000000    0.0  0.00000

# Reduce dimensions with LSA

In [14]:
reviews_lsa, lsa_components = reduce_dim_lsa(reviews_sparse_matrix_lsa, 14)

In [15]:
topic_names = ['Horror Comedy','Gory','Space Monster','Anime',
               'Silly','Animated Monster Concert','Kung Fu',
               'Music Documentary',"Creature Feature",'Foreign Monsters',
               'Sex Monsters','Camp','British',
               'Soviet']
#                 ,'Monsters On Drugs',
#                'Ridiculous', 'Holiday Monsters','Bollywood',
#                'Holiday Space Race','Holiday Monsters','Spaghetti Western']#, 21, 22,'Foreign Silliness', 'British']
display_topics(lsa_components, reviews_feature_names_lsa, 15, topic_names=topic_names)


Topic: ' Horror Comedy '
horror, comedy, funny, woman, music, action, girl, effect, kill, fun, laugh, sex, monster, kid, friend

Topic: ' Gory '
horror, vampire, zombie, monster, gore, alien, dracula, creature, effect, blood, scary, sci_fi, halloween, creepy, slasher

Topic: ' Space Monster '
alien, monster, animation, sci_fi, planet, space, science, creature, effect, ear, robot, scientist, giant, godzilla, ship

Topic: ' Anime '
vampire, animation, anime, dracula, disney, planet, cartoon, japanese, art, sci_fi, action, martial, hammer, version, fantasy

Topic: ' Silly '
comedy, vampire, funny, laugh, joke, hilarious, humor, fun, monster, gag, dracula, song, alien, funniest, kid

Topic: ' Animated Monster Concert '
animation, zombie, horror, band, anime, song, music, disney, cartoon, rock, concert, punk, musical, beatle, gore

Topic: ' Kung Fu '
martial, action, kung_fu, fight, japanese, kong, hong, dragon, zombie, anime, kill, animation, cop, violence, chinese

Topic: ' Music Documen

In [16]:
# reviews_sparse_matrix_lsa, reviews_feature_names_lsa = vectorize_tfidf(reviews, min_df=7, max_df=0.8)
# reviews_lsa, lsa_components = reduce_dim_lsa(reviews_sparse_matrix_lsa, 14)
# topic_names = ['Horror Comedy','Gory','Space Monster','Anime',
#                'Silly','Animated Monster Concert','Kung Fu',
#                'Music Documentary',"Creature Feature",'Foreign Monsters',
#                'Sex Monsters','Camp','British',
#                'Soviet']
# #                 ,'Monsters On Drugs',
# #                'Ridiculous', 'Holiday Monsters','Bollywood',
# #                'Holiday Space Race','Holiday Monsters','Spaghetti Western']#, 21, 22,'Foreign Silliness', 'British']
# display_topics(lsa_components, reviews_feature_names_lsa, 15, topic_names=topic_names)

# Topic: ' Horror Comedy '
# horror, comedy, funny, woman, music, action, girl, effect, kill, fun, laugh, sex, monster, kid, friend

# Topic: ' Gory '
# horror, vampire, zombie, monster, gore, alien, dracula, creature, effect, blood, scary, sci_fi, halloween, creepy, slasher

# Topic: ' Space Monster '
# alien, monster, sci_fi, animation, planet, space, science, creature, effect, ear, scientist, robot, giant, godzilla, ship

# Topic: ' Anime '
# vampire, animation, anime, dracula, disney, planet, cartoon, japanese, art, sci_fi, action, martial, hammer, version, fantasy

# Topic: ' Silly '
# comedy, vampire, funny, laugh, joke, hilarious, humor, fun, monster, gag, dracula, song, funniest, kid, alien

# Topic: ' Animated Monster Concert '
# animation, zombie, horror, band, anime, song, music, disney, cartoon, rock, concert, punk, musical, gore, cat

# Topic: ' Kung Fu '
# martial, action, kung_fu, fight, japanese, kong, hong, dragon, zombie, anime, kill, cop, chinese, animation, violence

# Topic: ' Music Documentary '
# band, music, rock, concert, punk, song, monster, martial, drug, beatle, documentary, musical, album, gang, footage

# Topic: ' Creature Feature '
# zombie, alien, russian, sci_fi, planet, soviet, band, vampire, space, polish, russia, science, ear, punk, fiction

# Topic: ' Foreign Monsters '
# monster, russian, soviet, zombie, godzilla, polish, martial, russia, japanese, frankenstein, poland, kung_fu, art, dracula, french

# Topic: ' Sex Monsters '
# sex, zombie, gay, porn, sexual, monster, woman, japanese, french, martial, erotic, art, rape, kung_fu, godzilla

# Topic: ' Camp '
# zombie, monster, indian, noir, western, dracula, gang, disney, animation, godzilla, bollyd, india, gangster, frankenstein, family

# Topic: ' British '
# russian, soviet, russia, monster, animation, anime, gang, sex, punk, rape, violence, cop, western, moscow, woman

# Topic: ' Soviet '
# school, girl, teen, kid, martial, russian, zombie, child, teacher, kung_fu, christmas, werewolf, boy, soviet, parent


In [17]:
# Topic: ' Horror Comedy '
# horror, comedy, funny, world, woman, music, performance, action, girl, effect, kill, fun, guy, young, laugh

# Topic: ' Gory '
# horror, vampire, monster, zombie, alien, gore, dracula, creature, effect, blood, scary, sci_fi, frankenstein, halloween, scare

# Topic: ' Space Monster '
# alien, monster, animation, sci_fi, planet, space, science, creature, godzilla, effect, giant, ear, robot, scientist, ship

# Topic: ' Anime '
# vampire, animation, anime, dracula, japanese, art, disney, planet, cartoon, world, beautiful, miyazaki, sci_fi, visual, action

# Topic: ' Silly '
# vampire, comedy, funny, laugh, dracula, joke, hilarious, humor, fun, gag, song, monster, funniest, kid, sketch

# Topic: ' Monster Concert '
# animation, zombie, horror, band, disney, anime, song, music, cartoon, rock, concert, punk, gore, musical, cat

# Topic: ' Kung Fu '
# martial, action, kung_fu, fight, japanese, kong, hong, dragon, kill, anime, violence, cop, gang, killer, chinese

# Topic: ' Music Documentary '
# band, music, rock, concert, monster, song, punk, beatle, drug, documentary, musical, album, footage, martial, musician

# Topic: ' Creature Feature '
# monster, godzilla, frankenstein, creature, giant, animation, kid, dracula, beast, japanese, child, girl, school, baron, toho

# Topic: ' Soviet Monsters '
# zombie, monster, russian, soviet, polish, martial, godzilla, french, japanese, poland, kung_fu, russia, band, art, comedy

# Topic: ' Sex Monsters '
# sex, gay, porn, zombie, sexual, japanese, woman, rape, anime, monster, alien, erotic, drug, martial, godzilla

# Topic: ' Camp '
# zombie, animation, monster, noir, cop, western, frankenstein, gangster, dracula, gang, crime, police, godzilla, drug, lugosi

# Topic: ' Schoolkid Martial Artist '
# zombie, school, girl, kid, teen, child, martial, disney, book, family, boy, young, moer, christmas, kung_fu

# Topic: ' Schoolkid Soviet Gangster '
# russian, soviet, russia, gang, girl, school, sex, rape, teen, moscow, western, polish, woman, kid, violence

# Reduce Dimensions with LDA

In [18]:
reviews_sparse_matrix_lda, reviews_feature_names_lda = vectorize_tfidf(reviews, min_df=7, max_df=0.8)

       aa       aan  aaron        ab  aback   abandon  abandonment     abba  \
0     0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
1     0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
2     0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
3     0.0  0.000000    0.0  0.000000    0.0  0.004250          0.0  0.00000   
4     0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
5     0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
6     0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
7     0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
8     0.0  0.000000    0.0  0.016791    0.0  0.007285          0.0  0.00000   
9     0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
10    0.0  0.000000    0.0  0.000000    0.0  0.000000          0.0  0.00000   
11    0.0  0.000000    0.0  0.000000    0.0  0.00000